# Estimation de la dérivée spatiale

Ce problème est étudié dans la section 6.2 du rapport.

In [2]:
import numpy as np 
import matplotlib.pyplot as plt 
import torch 
from utils.data_processing import *
from utils.vizualisation import *